In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# ESPN

## Process Match Results

In [ ]:
url = 'https://www.espncricinfo.com/records/tournament/team-match-results/icc-men-s-t20-world-cup-2022-23-14450'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

In [ ]:
table = soup.find_all('table')[0]
# table.find_all('td class')

In [ ]:
col_name = ['Team 1', 'Team 2','Winner','Margin','Ground','Match Date','Scorecard']
df_match = pd.DataFrame(columns = col_name)

In [ ]:
data_cols = table.find_all('tr')
for row in data_cols[1:]:
  row_data = [title.text.strip() for title in row.find_all('td')]
  ind = len(df_match)
  df_match.loc[ind] = row_data

In [ ]:
df_match.head()

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Scorecard
0,England,Pakistan,England,5 wickets,Melbourne,"Nov 13, 2022",T20I # 1879
1,England,India,England,10 wickets,Adelaide,"Nov 10, 2022",T20I # 1878
2,New Zealand,Pakistan,Pakistan,7 wickets,Sydney,"Nov 9, 2022",T20I # 1877
3,India,Zimbabwe,India,71 runs,Melbourne,"Nov 6, 2022",T20I # 1873
4,Bangladesh,Pakistan,Pakistan,5 wickets,Adelaide,"Nov 6, 2022",T20I # 1872


In [ ]:
df_match.shape

(42, 7)

In [ ]:
df_match.rename({'Scorecard':'Match_id'}, axis = 1, inplace = True)
df_match.head()

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match_id
0,England,Pakistan,England,5 wickets,Melbourne,"Nov 13, 2022",T20I # 1879
1,England,India,England,10 wickets,Adelaide,"Nov 10, 2022",T20I # 1878
2,New Zealand,Pakistan,Pakistan,7 wickets,Sydney,"Nov 9, 2022",T20I # 1877
3,India,Zimbabwe,India,71 runs,Melbourne,"Nov 6, 2022",T20I # 1873
4,Bangladesh,Pakistan,Pakistan,5 wickets,Adelaide,"Nov 6, 2022",T20I # 1872


Export to csv

In [ ]:
from google.colab import drive
import json
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/T20_WorldCup"

In [ ]:
df_match.to_csv(path+'/output/match.csv', index = False)

Read data from match.csv

In [ ]:
temp = pd.read_csv(path+'/output/match.csv')
temp.head()

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match_id
0,England,Pakistan,England,5 wickets,Melbourne,"Nov 13, 2022",T20I # 1879
1,England,India,England,10 wickets,Adelaide,"Nov 10, 2022",T20I # 1878
2,New Zealand,Pakistan,Pakistan,7 wickets,Sydney,"Nov 9, 2022",T20I # 1877
3,India,Zimbabwe,India,71 runs,Melbourne,"Nov 6, 2022",T20I # 1873
4,Bangladesh,Pakistan,Pakistan,5 wickets,Adelaide,"Nov 6, 2022",T20I # 1872


## Process Batting Summary

In [ ]:
with open(path+'/t20_wc_batting_summary.json') as f:
  data = json.load(f)

  all_records = []
  for rec in data:
    all_records.extend(rec['battingSummary'])

In [ ]:
df_batting = pd.DataFrame(all_records)

Change the 'dismissal' column into 'out/not out' column

*   If the cell is blank, this mean the player is not out and vice versa



In [ ]:
df_batting['dismissal'] = df_batting['dismissal'].apply(lambda x: 'not_out' if x == '' else 'out')

Remove special characters in batsmanName

In [ ]:
def replace_special_character(x):
  if '(c)' in x:
    x = x.replace('(c)','')
  if '†' in x:
    x = x.replace('†','')

  return x

In [ ]:
df_batting['batsmanName'] = df_batting['batsmanName'].apply(lambda x: replace_special_character(x))

Link df_batting and df_match together

In [ ]:
df_batting.head(10)

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,out,3,6,0,0,50.00
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,out,9,9,1,0,100.00
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,out,20,12,1,2,166.66
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,out,26,24,2,0,108.33
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus,out,20,24,0,0,83.33
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,out,44,28,4,0,157.14
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,out,0,1,0,0,0.00
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,not_out,31,16,2,2,193.75
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,out,9,10,1,0,90.00
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,out,6,6,0,0,100.00


In [ ]:
df_match.tail(5)

,Team 1,Team 2,Winner,Margin,Ground,Match Date,Match_id
37,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830
38,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
39,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
40,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
41,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823


In [ ]:
match_id_dict = {}
for index, row in df_match.iterrows():
  key1 = row['Team 1'] + ' Vs ' + row['Team 2']
  key2 = row['Team 2'] + ' Vs ' + row['Team 1']

  match_id_dict[key1] = row['Match_id']
  match_id_dict[key2] = row['Match_id']

In [ ]:
df_batting['Match_id'] = df_batting['match'].map(match_id_dict)

In [ ]:
df_batting.head()

,match,teamInnings,battingPos,batsmanName,dismissal,runs,balls,4s,6s,SR,Match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,out,3,6,0,0,50.00,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,out,9,9,1,0,100.00,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,out,20,12,1,2,166.66,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,out,26,24,2,0,108.33,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus,out,20,24,0,0,83.33,T20I # 1823


In [ ]:
df_batting.to_csv(path+'/output/fact_batting_summary.csv', index = False)

## Process Bowling Summary

In [ ]:
with open(path+'/t20_wc_bowling_summary.json') as f:
  data = json.load(f)

  all_records = []
  for rec in data:
    all_records.extend(rec['bowlingSummary'])

In [ ]:
df_bowling = pd.DataFrame(all_records)
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0


In [ ]:
df_bowling.shape

(500, 13)

Link df_bowling and df_match together

In [ ]:
df_bowling['Match_id'] = df_bowling['match'].map(match_id_dict)

In [ ]:
df_bowling.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4,0,27,1,6.75,8,1,1,0,0,T20I # 1823


Export to csv

In [ ]:
df_bowling.to_csv(path+'/output/fact_bowling_summary.csv', index= False)

## Process Players Information

In [ ]:
with open(path+'/t20_wc_player_info.json') as f:
  data = json.load(f)

In [ ]:
df_players = pd.DataFrame(data)
df_players.head()

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus(c),Namibia,Right hand Bat,Right arm Offbreak,Allrounder,


In [ ]:
df_players.shape

(219, 6)

Remove special characters

In [ ]:
df_players['name'] = df_players['name'].apply(lambda x: replace_special_character(x))

In [ ]:
df_players.head(20)

,name,team,battingStyle,bowlingStyle,playingRole,description
0,Michael van Lingen,Namibia,Left hand Bat,Left arm Medium,Bowling Allrounder,
1,Divan la Cock,Namibia,Right hand Bat,Legbreak,Opening Batter,
2,Jan Nicol Loftie-Eaton,Namibia,Left hand Bat,"Right arm Medium, Legbreak",Batter,
3,Stephan Baard,Namibia,Right hand Bat,Right arm Medium fast,Batter,
4,Gerhard Erasmus,Namibia,Right hand Bat,Right arm Offbreak,Allrounder,
5,Jan Frylinck,Namibia,Left hand Bat,Left arm Fast medium,Allrounder,
6,David Wiese,Namibia,Right hand Bat,Right arm Medium fast,Allrounder,David Wiese joined a marked outflow of South A...
7,JJ Smit,Namibia,Right hand Bat,Left arm Medium fast,Bowling Allrounder,
8,Pathum Nissanka,Sri Lanka,Right hand Bat,,Top order Batter,
9,Kusal Mendis,Sri Lanka,Right hand Bat,Legbreak,Wicketkeeper Batter,"Blessed with a compact technique, an aggressiv..."


Export to csv

In [ ]:
df_players.to_csv(path+'/output/dim_players_no_images.csv', index = False)